<a href="https://colab.research.google.com/github/kp8757/Trading_Bot_Demo/blob/main/Last_Advanced_Binance_Futures_Trading_Bot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install python-binance

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.8/130.8 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 315.5/315.5 kB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 49.7 MB/s eta 0:00:00


In [ ]:
!pip install binance-futures-connector

In [ ]:
import os
import logging
import time
import threading
from collections import deque # To store a limited number of recent prices/candles


In [ ]:

# UPDATED IMPORTS FOR COMPATIBILITY WITH SOME PYTHON-BINANCE VERSIONS
from binance.um_futures import UMFutures
from binance.websocket.um_futures.websocket_client import UMFuturesWebsocketClient
from binance.lib.utils import config_logging
from binance.error import ClientError


In [ ]:
# Configure logging for better visibility
config_logging(logging, logging.INFO) # Changed to INFO for less verbosity in console by default

class BasicBot:
    """
    An advanced trading bot for Binance Futures Testnet (USDT-M).
    Features:
    - Market, Limit, and Stop-Market order placement.
    - Real-time price data streaming via WebSockets.
    - Simple Moving Average (SMA) crossover strategy.
    - Basic position tracking.
    - Robust logging and error handling.
    """
    def __init__(self, api_key: str, api_secret: str, testnet: bool = True):
        """
        Initializes the bot with API credentials, sets up the client,
        and prepares for WebSocket connections and strategy execution.

        Args:
            api_key (str): Your Binance API key.
            api_secret (str): Your Binance API secret.
            testnet (bool): True to use Binance Testnet, False for production.
        """
        self.api_key = "your api key"
        self.api_secret = "your secret key"
        self.testnet = testnet
        self.base_url = "https://testnet.binancefuture.com" if testnet else "https://fapi.binance.com"
        self.ws_base_url = "wss://fstream.binance.com" if not testnet else "wss://stream.binancefuture.com"

        self.client = self._initialize_client()
        self.ws_client = None
        self.current_price = {} # Stores the latest price for subscribed symbols
        self.klines_data = {}   # Stores kline data for strategy calculation {symbol: deque of klines}
        self.strategy_active = False
        self.position_info = {} # Stores current open positions {symbol: {positionAmt, entryPrice, ...}}
        self.order_placement_lock = threading.Lock() # To prevent concurrent order placements

        logging.info(f"Bot initialized. Using Testnet: {self.testnet}, Base URL: {self.base_url}")

    def _initialize_client(self):
        """
        Initializes and returns the Binance UMFutures client.
        Tests connection to ensure API keys are valid.
        """
        try:
            client = UMFutures(key=self.api_key, secret=self.api_secret, base_url=self.base_url)
            client.time() # Test connection
            logging.info("Successfully connected to Binance REST API.")
            return client
        except ClientError as e:
            logging.error(f"Failed to connect to Binance REST API: {e}")
            raise ConnectionError("Could not connect to Binance REST API. Check your API keys and network.")
        except Exception as e:
            logging.error(f"An unexpected error occurred during REST client initialization: {e}")
            raise

    def get_account_balance(self):
        """
        Fetches and prints the account balance.
        Updates self.position_info with current open positions.
        """
        try:
            account_info = self.client.account()
            logging.info("--- Account Balance ---")
            for asset in account_info['assets']:
                if float(asset['walletBalance']) > 0:
                    logging.info(f"Asset: {asset['asset']}, Wallet Balance: {asset['walletBalance']}, "
                                 f"Cross Wallet Balance: {asset['crossWalletBalance']}")

            self.position_info = {} # Clear previous positions
            logging.info("--- Current Positions ---")
            for position in account_info['positions']:
                if float(position['positionAmt']) != 0:
                    symbol = position['symbol']
                    self.position_info[symbol] = {
                        'positionAmt': float(position['positionAmt']),
                        'entryPrice': float(position['entryPrice']),
                        'unRealizedProfit': float(position['unRealizedProfit']),
                        'leverage': int(position['leverage']),
                        'isolatedWallet': float(position['isolatedWallet']) if position['isolatedWallet'] else 0.0
                    }
                    logging.info(f"Symbol: {symbol}, Position: {self.position_info[symbol]['positionAmt']},"
                                 f" Entry Price: {self.position_info[symbol]['entryPrice']},"
                                 f" Unrealized PnL: {self.position_info[symbol]['unRealizedProfit']:.2f}")
            if not self.position_info:
                logging.info("No open positions.")
            return account_info
        except ClientError as e:
            logging.error(f"Error getting account balance or positions: {e}")
            return None
        except Exception as e:
            logging.error(f"An unexpected error occurred while getting balance/positions: {e}")
            return None

    def place_order(self, symbol: str, side: str, order_type: str, quantity: float, price: float = None, stop_price: float = None):
        """
        Places an order on Binance Futures Testnet.
        Uses a lock to prevent multiple orders from being placed simultaneously.

        Args:
            symbol (str): Trading pair (e.g., 'BTCUSDT').
            side (str): 'BUY' or 'SELL'.
            order_type (str): 'MARKET', 'LIMIT', or 'STOP_MARKET'.
            quantity (float): Amount of base asset to trade.
            price (float, optional): Required for 'LIMIT' orders.
            stop_price (float, optional): Required for 'STOP_MARKET' orders.

        Returns:
            dict: The order response from Binance, or None if an error occurred.
        """
        with self.order_placement_lock:
            try:
                params = {
                    'symbol': symbol,
                    'side': side.upper(),
                    'type': order_type.upper(),
                    'quantity': quantity,
                }

                if order_type.upper() == 'LIMIT':
                    if price is None:
                        raise ValueError("Price is required for LIMIT orders.")
                    params['price'] = price
                    params['timeInForce'] = 'GTC' # Good Till Cancelled

                elif order_type.upper() == 'STOP_MARKET':
                    if stop_price is None:
                        raise ValueError("Stop price is required for STOP_MARKET orders.")
                    params['stopPrice'] = stop_price
                    params['closePosition'] = 'false'
                    params['newOrderRespType'] = 'ACK'

                logging.info(f"Attempting to place order: {params}")
                order = self.client.new_order(**params)
                logging.info(f"Order placed successfully: {order}")
                return order
            except ClientError as e:
                logging.error(f"Error placing order: {e}")
                return None
            except ValueError as e:
                logging.error(f"Validation error for order parameters: {e}")
                return None
            except Exception as e:
                logging.error(f"An unexpected error occurred while placing order: {e}")
                return None

    def _on_message(self, ws, message):
        """
        Callback function for WebSocket messages.
        Processes real-time kline data and updates current price.
        """
        try:
            event = message
            if 'e' in event and event['e'] == 'kline':
                kline = event['k']
                symbol = kline['s']
                interval = kline['i']
                is_kline_closed = kline['x']
                close_price = float(kline['c'])

                # Initialize deque for symbol if not exists
                if symbol not in self.klines_data:
                    self.klines_data[symbol] = deque(maxlen=200) # Store up to 200 klines

                # Only add closed klines for SMA calculation
                if is_kline_closed:
                    self.klines_data[symbol].append(close_price)
                    logging.debug(f"Received closed kline for {symbol} ({interval}): {close_price}")

                self.current_price[symbol] = close_price # Always update current price
                # print(f"Real-time Price for {symbol}: {close_price}") # For immediate CLI feedback
                self.execute_strategy(symbol) # Execute strategy on new kline data

            elif 'e' in event and event['e'] == 'markPriceUpdate':
                symbol = event['s']
                mark_price = float(event['p'])
                self.current_price[symbol] = mark_price
                # logging.debug(f"Received mark price for {symbol}: {mark_price}")

            # You can add more event types here (e.g., user data streams for order/position updates)
        except Exception as e:
            logging.error(f"Error processing WebSocket message: {e}, Message: {message}")

    def _on_open(self, ws):
        logging.info("WebSocket connection opened.")

    def _on_close(self, ws):
        logging.warning("WebSocket connection closed.")

    def _on_error(self, ws, error):
        logging.error(f"WebSocket error: {error}")

    def start_websocket_stream(self, symbols: list, interval: str = '1m'):
        """
        Starts a WebSocket stream for kline data for specified symbols.

        Args:
            symbols (list): List of trading pairs (e.g., ['BTCUSDT', 'ETHUSDT']).
            interval (str): Kline interval (e.g., '1m', '5m', '1h').
        """
        if self.ws_client:
            logging.warning("WebSocket client already running. Please stop it first.")
            return

        streams = [f"{s.lower()}@kline_{interval}" for s in symbols]
        # Alternatively, for just price updates, use markPrice stream:
        # streams = [f"{s.lower()}@markPrice@1s" for s in symbols]

        self.ws_client = UMFuturesWebsocketClient(on_message=self._on_message,
                                                  on_open=self._on_open,
                                                  on_close=self._on_close,
                                                  on_error=self._on_error)
        self.ws_client.continuous_subscribe(streams) # Corrected method
        logging.info(f"Subscribed to WebSocket streams: {streams}")

    def stop_websocket_stream(self):
        """
        Stops the active WebSocket stream.
        """
        if self.ws_client:
            # The UMFuturesWebsocketClient doesn't have a simple 'stop' or 'close' method
            # like some other libraries. We can try to unsubscribe.
            # streams = [f"{s.lower()}@kline_{interval}" for s in self.current_price.keys()] # Need to remember symbols/interval
            # self.ws_client.unsubscribe(streams) # This method might not be directly available or work as expected for continuous
            logging.warning("Stopping WebSocket stream might require restarting the kernel for a clean closure with this library version.")
            # A more robust way to truly stop is often to manage the connection thread directly,
            # but the client doesn't expose it easily. For now, rely on garbage collection
            # or kernel restart, or explore the library's internal methods if needed.
            # As a workaround, set the client to None to indicate it's stopped from bot's perspective.
            self.ws_client = None
            logging.info("WebSocket client reference cleared. Stream might continue until connection times out or kernel restarts.")
        else:
            logging.info("No active WebSocket connection to stop.")

    def calculate_sma(self, symbol: str, period: int):
        """
        Calculates the Simple Moving Average (SMA) for a given symbol and period.
        Requires sufficient kline data to be available.

        Args:
            symbol (str): Trading pair.
            period (int): The period for SMA calculation.

        Returns:
            float: The calculated SMA, or None if not enough data.
        """
        if symbol not in self.klines_data or len(self.klines_data[symbol]) < period:
            logging.warning(f"Not enough kline data for {symbol} to calculate SMA({period}). "
                            f"Need {period}, have {len(self.klines_data.get(symbol, []))}.")
            return None

        # Take the last 'period' number of prices from the deque
        prices = list(self.klines_data[symbol])[-period:]
        sma = sum(prices) / len(prices)
        return sma

    def execute_strategy(self, symbol: str):
        """
        Implements a simple SMA crossover strategy.
        - Buys when short SMA crosses above long SMA.
        - Sells/closes position when short SMA crosses below long SMA.
        """
        if not self.strategy_active:
            return

        short_sma_period = 10 # Example short period
        long_sma_period = 30  # Example long period

        short_sma = self.calculate_sma(symbol, short_sma_period)
        long_sma = self.calculate_sma(symbol, long_sma_period)

        if short_sma is None or long_sma is None:
            return # Not enough data yet

        current_position_amt = self.position_info.get(symbol, {}).get('positionAmt', 0.0)

        logging.debug(f"Strategy check for {symbol}: Short SMA({short_sma_period}): {short_sma:.2f}, "
                      f"Long SMA({long_sma_period}): {long_sma:.2f}, Current Price: {self.current_price.get(symbol, 'N/A')}")

        # Buy Signal: Short SMA crosses above Long SMA
        # Only buy if not already in a long position or if shorting is allowed and we are in a short position
        if short_sma > long_sma and current_position_amt <= 0: # Check if not already long or neutral
            logging.info(f"BUY Signal for {symbol}! Short SMA ({short_sma:.2f}) > Long SMA ({long_sma:.2f})")
            # For simplicity, we'll always buy a fixed quantity. In a real bot, use position sizing.
            order_quantity = 0.001 # Example quantity for BTCUSDT
            if symbol == 'ETHUSDT': order_quantity = 0.01 # Example quantity for ETHUSDT

            # Ensure we don't try to open multiple long positions if already long
            if current_position_amt < 0: # If currently short, close short first
                logging.info(f"Closing short position ({abs(current_position_amt)}) for {symbol} before going long.")
                self.place_order(symbol, 'BUY', 'MARKET', abs(current_position_amt))
                time.sleep(1) # Give a moment for order to process
                self.get_account_balance() # Refresh position info

            if self.position_info.get(symbol, {}).get('positionAmt', 0.0) == 0: # If still neutral after closing short
                self.place_order(symbol, 'BUY', 'MARKET', order_quantity)
                self.get_account_balance() # Refresh position info after placing order
            else:
                logging.info(f"Already in a position for {symbol}, not placing new BUY order.")


        # Sell Signal: Short SMA crosses below Long SMA
        # Only sell if not already in a short position or if longing is allowed and we are in a long position
        elif short_sma < long_sma and current_position_amt >= 0: # Check if not already short or neutral
            logging.info(f"SELL Signal for {symbol}! Short SMA ({short_sma:.2f}) < Long SMA ({long_sma:.2f})")
            order_quantity = 0.001 # Example quantity for BTCUSDT
            if symbol == 'ETHUSDT': order_quantity = 0.01 # Example quantity for ETHUSDT

            # Ensure we don't try to open multiple short positions if already short
            if current_position_amt > 0: # If currently long, close long first
                logging.info(f"Closing long position ({abs(current_position_amt)}) for {symbol} before going short.")
                self.place_order(symbol, 'SELL', 'MARKET', abs(current_position_amt))
                time.sleep(1) # Give a moment for order to process
                self.get_account_balance() # Refresh position info

            if self.position_info.get(symbol, {}).get('positionAmt', 0.0) == 0: # If still neutral after closing long
                self.place_order(symbol, 'SELL', 'MARKET', order_quantity)
                self.get_account_balance() # Refresh position info after placing order
            else:
                logging.info(f"Already in a position for {symbol}, not placing new SELL order.")

def validate_input(prompt: str, type_func, min_val=None, max_val=None):
    """
    Helper function to get and validate user input.
    """
    while True:
        try:
            value = type_func(input(prompt))
            if min_val is not None and value < min_val:
                print(f"Value must be at least {min_val}.")
                continue
            if max_val is not None and value > max_val:
                print(f"Value must be at most {max_val}.")
                continue
            return value
        except ValueError:
            print("Invalid input. Please enter a valid number.")

def main():
    """
    Main function to run the trading bot CLI with advanced features.
    """
    # Directly use the provided API keys
    api_key ="your api key"
    api_secret ="your api secret key"

    # Fallback to environment variables if the hardcoded ones are empty (though they aren't in this case)
    if not api_key:
        api_key = os.getenv("BINANCE_API_KEY")
    if not api_secret:
        api_secret = os.getenv("BINANCE_API_SECRET")

    if not api_key or not api_secret:
        print("Error: API_KEY and API_SECRET are missing. Please provide them or set environment variables.")
        return

    bot = None
    try:
        bot = BasicBot(api_key, api_secret, testnet=True)
        bot.get_account_balance() # Show initial balance and positions

        # Start a background thread to display real-time prices
        def price_display_thread(bot_instance):
            while True:
                if bot_instance.current_price:
                    price_str = ", ".join([f"{s}: {p:.2f}" for s, p in bot_instance.current_price.items()])
                    position_str = ", ".join([f"{s}: {pos['positionAmt']:.3f} (PnL: {pos['unRealizedProfit']:.2f})"
                                              for s, pos in bot_instance.position_info.items()])
                    print(f"\rLive Prices: {price_str} | Positions: {position_str}    ", end="", flush=True)
                time.sleep(1)

        display_thread = threading.Thread(target=price_display_thread, args=(bot,), daemon=True)
        display_thread.start()

        while True:
            print("\n\n--- Trading Bot Menu ---")
            print("1. Place Market Order")
            print("2. Place Limit Order")
            print("3. Place Stop-Market Order")
            print("4. Check Account Balance & Positions")
            print("5. Start Real-time Data Stream (for strategy)")
            print("6. Stop Real-time Data Stream")
            print("7. Toggle Strategy (SMA Crossover)")
            print("8. Exit")

            choice = input("Enter your choice: ")

            if choice == '1':
                symbol = input("Enter symbol (e.g., BTCUSDT): ").upper()
                side = input("Enter side (BUY/SELL): ").upper()
                quantity = validate_input("Enter quantity: ", float, min_val=0.001)
                order_response = bot.place_order(symbol, side, 'MARKET', quantity)
                if order_response:
                    print(f"Market order placed. Order ID: {order_response.get('orderId')}, Status: {order_response.get('status')}")

            elif choice == '2':
                symbol = input("Enter symbol (e.g., BTCUSDT): ").upper()
                side = input("Enter side (BUY/SELL): ").upper()
                quantity = validate_input("Enter quantity: ", float, min_val=0.001)
                price = validate_input("Enter limit price: ", float, min_val=0.01)
                order_response = bot.place_order(symbol, side, 'LIMIT', quantity, price=price)
                if order_response:
                    print(f"Limit order placed. Order ID: {order_response.get('orderId')}, Status: {order_response.get('status')}")

            elif choice == '3':
                symbol = input("Enter symbol (e.g., BTCUSDT): ").upper()
                side = input("Enter side (BUY/SELL): ").upper()
                quantity = validate_input("Enter quantity: ", float, min_val=0.001)
                stop_price = validate_input("Enter stop price: ", float, min_val=0.01)
                order_response = bot.place_order(symbol, side, 'STOP_MARKET', quantity, stop_price=stop_price)
                if order_response:
                    print(f"Stop-Market order placed. Order ID: {order_response.get('orderId')}, Status: {order_response.get('status')}")

            elif choice == '4':
                bot.get_account_balance()

            elif choice == '5':
                symbols_input = input("Enter symbols to stream (comma-separated, e.g., BTCUSDT,ETHUSDT): ").upper()
                symbols_list = [s.strip() for s in symbols_input.split(',')]
                interval = input("Enter kline interval (e.g., 1m, 5m, 1h): ")
                bot.start_websocket_stream(symbols_list, interval)

            elif choice == '6':
                bot.stop_websocket_stream()

            elif choice == '7':
                bot.strategy_active = not bot.strategy_active
                status = "ACTIVE" if bot.strategy_active else "INACTIVE"
                print(f"SMA Crossover Strategy Toggled to: {status}")
                if bot.strategy_active and not bot.ws_client:
                    print("Warning: Strategy is active but no WebSocket stream is running. Start stream first.")

            elif choice == '8':
                print("Exiting bot. Goodbye!")
                if bot:
                    bot.stop_websocket_stream()
                break
            else:
                print("Invalid choice. Please try again.")

    except ConnectionError as e:
        print(f"Bot could not start due to connection error: {e}")
    except Exception as e:
        logging.exception("An unhandled error occurred in main execution:")
        print(f"An unhandled error occurred: {e}. Check logs for details.")
    finally:
        # The UMFuturesWebsocketClient doesn't have a simple 'stop' or 'close' method
        # as noted in the stop_websocket_stream method. Cleaning up might require
        # a kernel restart for a full stop.
        logging.info("Bot finished. WebSocket stream might remain active until kernel restart.")

if __name__ == "__main__":
    main()

ERROR:root:Error getting account balance or positions: (401, -2015, 'Invalid API-key, IP, or permissions for action', {'Content-Type': 'application/json', 'Content-Length': '69', 'Connection': 'keep-alive', 'Date': 'Fri, 01 Aug 2025 10:23:18 GMT', 'Access-Control-Allow-Methods': 'GET, POST, PUT, DELETE, OPTIONS', 'Server': 'Bengine', 'x-mbx-used-weight-1m': '5', 'x-response-time': '2ms', 'Access-Control-Allow-Origin': '*', 'X-Cache': 'Error from cloudfront', 'Via': '1.1 03aeebadea872027865b9f27d58208c0.cloudfront.net (CloudFront)', 'X-Amz-Cf-Pop': 'LAX50-P1', 'X-Amz-Cf-Id': 'kAg1ow_Xrw8FK24NuwbffF1vNCrnM0R-WilSTffYmoe4myaSU17pBA=='})




--- Trading Bot Menu ---
1. Place Market Order
2. Place Limit Order
3. Place Stop-Market Order
4. Check Account Balance & Positions
5. Start Real-time Data Stream (for strategy)
6. Stop Real-time Data Stream
7. Toggle Strategy (SMA Crossover)
8. Exit
